In [1]:
from h3 import h3
import s2
import s2geometry

In [2]:
from pymongo import MongoClient
from keplergl import KeplerGl
from shapely.geometry import Polygon, shape, mapping
import geopandas as gpd

In [3]:
client = MongoClient('mongodb://localhost:27017/')
db = client.osmDataDB
coll_areas = db.areas

In [4]:
wroclaw = coll_areas.find_one({ 'osm_id': 2805691 })
wroclaw_shp = wroclaw['geometry']
wroclaw_geojson = {
    "type": "Feature",
    "properties": {"name": "Wrocław"},
    "geometry": wroclaw_shp
}

In [5]:
wroclaw_polygon = shape(wroclaw_shp)
buffered_polygon = wroclaw_polygon.buffer(0.005)
wroclaw_gdf = gpd.GeoDataFrame({'geometry': [buffered_polygon]}, crs="EPSG:3857")

In [6]:
type(buffered_polygon)

shapely.geometry.polygon.Polygon

In [7]:
polyline = wroclaw_shp['coordinates'][0]
polyline.append(polyline[0])

In [8]:
hexagons = list(h3.polyfill(mapping(buffered_polygon), 8))

In [19]:
hexagons = list(h3.polyfill(wroclaw_shp, 8))
hexagons_2 = list(h3.polyfill(wroclaw_shp, 11))

In [20]:
polylines = []
for hex in hexagons:
    polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
    # flatten polygons into loops.
    outlines = [loop for polygon in polygons for loop in polygon]
    polyline = [outline + [outline[0]] for outline in outlines][0]
    polylines.append(Polygon(polyline))
polylines_2 = []
for hex in hexagons_2:
    polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
    # flatten polygons into loops.
    outlines = [loop for polygon in polygons for loop in polygon]
    polyline = [outline + [outline[0]] for outline in outlines][0]
    polylines_2.append(Polygon(polyline))

In [21]:
h3_gdf = gpd.GeoDataFrame({'geometry': polylines}, crs="EPSG:3857")
h3_gdf_2 = gpd.GeoDataFrame({'geometry': polylines_2}, crs="EPSG:3857")

In [12]:
h3_gdf

,geometry
0,"POLYGON ((16.86391 51.16397, 16.86003 51.16581..."
1,"POLYGON ((17.02701 51.06919, 17.02706 51.07359..."
2,"POLYGON ((17.14866 51.09128, 17.15260 51.09384..."
3,"POLYGON ((16.84023 51.14427, 16.84029 51.14866..."
4,"POLYGON ((16.94898 51.08843, 16.94903 51.09283..."
...,...
729,"POLYGON ((17.13710 51.11002, 17.13320 51.11186..."
730,"POLYGON ((16.90297 51.16318, 16.89903 51.16063..."
731,"POLYGON ((17.03116 51.08935, 17.02721 51.08679..."
732,"POLYGON ((16.86703 51.10070, 16.86708 51.10509..."


In [23]:
h3_gdf_2

In [22]:
map_1 = KeplerGl(height=800)
map_1.add_data(data=wroclaw_geojson, name='feature_2')
# map_1.add_data(data=wroclaw_gdf, name='wroclaw')
map_1.add_data(data=h3_gdf, name='polylines')
map_1.add_data(data=h3_gdf_2, name='polylines_2')
# map_1.config = map_config
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'feature_2': {'type': 'Feature', 'properties': {'name': 'Wrocław'}, 'geometry': {'type': 'Polyg…

In [120]:
map_config = map_1.config

In [137]:
len(wroclaw_shp['coordinates'][0][0])

2

In [143]:
bbox = wroclaw_polygon.bounds

In [144]:
bbox

(16.8073393, 51.0426686, 17.1762192, 51.2100604)

In [145]:
rect = s2geometry.S2LatLngRect(
        s2geometry.S2LatLng.FromDegrees(bbox[1], bbox[0]),
        s2geometry.S2LatLng.FromDegrees(bbox[3], bbox[2]))

In [212]:
coverer = s2geometry.S2RegionCoverer()
coverer.set_min_level(14)
coverer.set_max_level(14)
coverer.set_max_cells(1000)

In [213]:
covering = coverer.GetCovering(rect)

In [214]:
s2polygons = []
for cellid in covering:
    new_cell = s2geometry.S2Cell(cellid)
    vertices = []
    for i in range(4):
        vertex = new_cell.GetVertex(i)
        latlng = s2geometry.S2LatLng(vertex)
        vertices.append((latlng.lng().degrees(),
                         latlng.lat().degrees()))
    geo = Polygon(vertices)
    s2polygons.append(geo)

In [215]:
s2_gdf = gpd.GeoDataFrame({'geometry': s2polygons}, crs="EPSG:3857")

In [216]:
s2_gdf

,geometry
0,"POLYGON ((16.83413 51.20596, 16.83719 51.21089..."
1,"POLYGON ((16.82689 51.20702, 16.82995 51.21196..."
2,"POLYGON ((16.81966 51.20809, 16.82271 51.21302..."
3,"POLYGON ((16.81242 51.20916, 16.81548 51.21409..."
4,"POLYGON ((16.80937 51.20423, 16.81242 51.20916..."
...,...
1686,"POLYGON ((17.15573 51.20104, 17.15884 51.20596..."
1687,"POLYGON ((17.15884 51.20596, 17.16196 51.21088..."
1688,"POLYGON ((17.16609 51.20487, 17.16921 51.20978..."
1689,"POLYGON ((17.16921 51.20978, 17.17232 51.21470..."


In [217]:
pip = s2_gdf.intersects(wroclaw_polygon)

In [218]:
ec_gdf = s2_gdf.loc[pip].copy()

In [219]:
ec_gdf

,geometry
47,"POLYGON ((16.88785 51.20339, 16.89091 51.20832..."
48,"POLYGON ((16.88478 51.19846, 16.88785 51.20339..."
49,"POLYGON ((16.89202 51.19739, 16.89508 51.20232..."
50,"POLYGON ((16.89508 51.20232, 16.89815 51.20724..."
51,"POLYGON ((16.89815 51.20724, 16.90122 51.21217..."
...,...
1567,"POLYGON ((17.01505 51.17907, 17.01814 51.18399..."
1568,"POLYGON ((17.01814 51.18399, 17.02122 51.18892..."
1569,"POLYGON ((17.02538 51.18291, 17.02847 51.18783..."
1603,"POLYGON ((16.96946 51.20741, 16.97254 51.21234..."


In [220]:
map_1 = KeplerGl(height=800)
map_1.add_data(data=wroclaw_geojson, name='feature_2')
# map_1.add_data(data=wroclaw_gdf, name='wroclaw')
map_1.add_data(data=ec_gdf, name='polylines')
map_1.config = map_config
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': '08zph6d', 'type': …